In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import tensorflow as tf
import matplotlib.pyplot as plt


In [11]:
#####################
##  PREPROCESSING  ##
#####################

In [12]:
from deep_lab.data_processing import create_tf_dataset, load_and_preprocess_data

# Preprocess data paths
image_dir = r'D:\01_Arnaud\Etudes\04_CNAM\RCP209\Projet\CodeV3\deep-lab3\data\VOCdevkit\VOC2012\JPEGImages'
mask_dir = r'D:\01_Arnaud\Etudes\04_CNAM\RCP209\Projet\CodeV3\deep-lab3\data\VOCdevkit\VOC2012\SegmentationClass'

train_images, val_images, train_masks, val_masks = load_and_preprocess_data(image_dir, mask_dir)

# Create TensorFlow datasets
train_dataset = create_tf_dataset(train_images, train_masks)
val_dataset = create_tf_dataset(val_images, val_masks)

In [13]:
############################
## TRAINING & FINE-TUNING ##
############################

In [38]:
from deep_lab3.learning_rate import PolyDecay
from deep_lab3.metrics import MeanIoU
from deep_lab3.model_gpt import DeepLabV3Plus
from tensorflow.keras import Model


initial_lr = 0.001
epochs = 50
poly_decay = PolyDecay(initial_learning_rate=initial_lr, max_epochs=epochs)
optimizer = tf.keras.optimizers.Adam(learning_rate=poly_decay, weight_decay=0.0005)

model = DeepLabV3Plus(dropout_rate=0.5)
# input_shape = (224, 224, 3)
# input = tf.keras.Input(shape=input_shape)
# model = Model(inputs=input, outputs=model.call(input))
# for layer in model.layers:
#     if not isinstance(layer, tf.keras.layers.BatchNormalization):
#         layer.trainable = False

# for layer in model.aspp.layers + model.decoder.layers:
#     layer.trainable = True

# model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy', MeanIoU(num_classes=NUM_CLASSES)])

# history = model.fit(train_dataset, validation_data=val_dataset, epochs=epochs)

for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy', MeanIoU()])


In [39]:
from tensorflow.keras.callbacks import EarlyStopping # type: ignore

# Définition de l'arrêt anticipé (early stopping)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5


ValueError: Exception encountered when calling Add.call().

[1mDimensions must be equal, but are 14 and 4 for '{{node DeepLabV3Plus_1/backbone_1/resnet101_1/conv4_block1_add_1/Add}} = AddV2[T=DT_FLOAT](DeepLabV3Plus_1/backbone_1/resnet101_1/conv4_block1_0_bn_1/batchnorm/add_1, DeepLabV3Plus_1/backbone_1/resnet101_1/conv4_block1_3_bn_1/batchnorm/add_1)' with input shapes: [?,14,14,1024], [?,4,4,1024].[0m

Arguments received by Add.call():
  • inputs=['tf.Tensor(shape=(None, 14, 14, 1024), dtype=float32)', 'tf.Tensor(shape=(None, 4, 4, 1024), dtype=float32)']

In [ ]:
# Optional: save the model
model.save('results/my_model.h5')

In [ ]:
##################
##  EVALUATION  ##
##################

In [ ]:

model.evaluate(val_dataset)

In [ ]:
########################
## INFERENCE EXEMPLE  ## 
########################

In [ ]:
# Display result for one image
val_images, val_masks = next(iter(val_dataset))
image = val_images[0]
prediction = model.predict(tf.expand_dims(image, axis=0))
predicted_mask = tf.argmax(prediction, axis=-1)
predicted_mask = tf.squeeze(predicted_mask)

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title("Input Image")
plt.imshow(image)

plt.subplot(1, 3, 2)
plt.title("True Mask")
plt.imshow(tf.squeeze(val_masks[0]), cmap='gray')

plt.subplot(1, 3, 3)
plt.title("Predicted Mask")
plt.imshow(predicted_mask, cmap='gray')

plt.show()

In [ ]:
## Evaluate on the offcial testin